<a href="https://colab.research.google.com/github/raviteja1432/NNDL/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
import math
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/loaddata.xlsx")
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [ ]:
x = []
y = []
for i in range(24, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24])
    y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous Day Load in kw' : x, 'Present Day Load in kw' : y})
data.head()

,Previous Day Load in kw,Present Day Load in kw
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


In [ ]:

#normalizing
maxX = np.max(data['Previous Day Load in kw'])
minX = np.min(data['Previous Day Load in kw'])
maxY = np.max(data['Present Day Load in kw'])
minY = np.min(data['Present Day Load in kw'])

data['Previous Day Load in kw'] = (data['Previous Day Load in kw'] - minX) / (maxX - minX)
data['Present Day Load in kw'] = (data['Present Day Load in kw']- minY) / (maxY - minY)

In [ ]:
from sklearn.model_selection import train_test_split
x = data['Previous Day Load in kw']
y = data['Present Day Load in kw']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [ ]:
#Model Training
#Initialization
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.5
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        # Calculating updated values of gm2 and gc2
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        
        # Updating m and c values
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.7303062534193249
c = 0.12533345571229326


In [ ]:


train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
rmse = math.sqrt(mean_squared_error(train_pred, trainY)) 
print("testing error")
print('Root mean square error', rmse) 
mse = (mean_squared_error(train_pred,trainY)) 
print('Mean square error', mse) 
mae=mean_absolute_error(train_pred,trainY)
print('Mean absolute error', mae)


#Testing Accuracies
rmse = math.sqrt(mean_squared_error(test_pred,testY ))
print("testing error") 
print('Root mean square error', rmse) 
mse = (mean_squared_error(test_pred,testY))
print('Mean square error', mse) 
mae=mean_absolute_error(test_pred,testY)
print('Mean absolute error', mae)

testing error
Root mean square error 718.5359132864205
Mean square error 516293.8586823505
Mean absolute error 529.3849582596877
testing error
Root mean square error 700.6816160368415
Mean square error 490954.72705199977
Mean absolute error 542.2870123812264


In [ ]:

#checking the difference between actal value and predicted value of training data
pd.DataFrame({'Actual value' : trainY, 'Predicted value' : train_pred})

,Actual value,Predicted value
0,4221.85392,4617.567090
1,7609.43808,6549.206565
2,5847.05880,5953.594959
3,4672.50300,5012.706212
4,7883.78688,6661.225261
...,...,...
1939,5830.37964,5709.749382
1940,6110.49600,5905.326740
1941,5289.94368,5471.823491
1942,7962.97392,7601.544808


In [ ]:


#checking the difference between actal value and predicted value of testing data
pd.DataFrame({'Actual value' : testY, 'Predicted value' : test_pred})

,Actual value,Predicted value
0,6161.624640,5702.235932
1,5761.792440,6953.566738
2,7390.270800,6441.969154
3,4597.368840,5879.826549
4,5242.712040,5995.260450
...,...,...
211,6764.319072,7551.796667
212,7390.270800,6776.659162
213,6550.077600,6592.238137
214,5722.043040,5596.478443


In [ ]:
#testing for real example
x = float(input('Enter the load of previous day at present hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load of previous day at present hour : 4983.1
Predicted load at present hour : 5234.983537594161
